In [1]:
# import libraries
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns 

# for making the output constant across all run
np.random.seed(42)

# display settings & code formatting
pd.options.display.max_columns = 999
%matplotlib inline

# project paths
# project_root_dir = os.path.normpath(os.getcwd() + os.sep + os.pardir)

# data_path = os.path.join(project_root_dir, "data")
# os.makedirs(data_path, exist_ok=True)

data_path = "/content/drive/MyDrive/workspace/walmart/data"

# function for loading data
def read_data(filename, date_col=None, data_path=data_path):
    csv_path = os.path.join(data_path, filename)
    return pd.read_csv(csv_path, parse_dates=date_col)

# function for saving data as csv file
def save_dataframe(df, filename, file_path=data_path):
    path = os.path.join(file_path, filename)
    df.to_csv(path, index=False)


## Prepare Data

In [2]:
# Read data
train = read_data("train.csv", date_col=["Date"])
test = read_data("test.csv", date_col=["Date"])
stores = read_data("stores.csv")
features = read_data("features.csv", date_col=["Date"])
sample_submission = read_data("sampleSubmission.csv")


# Merge the stores data with train and test
train = pd.merge(train, stores, how="left", on="Store")
test = pd.merge(test, stores, how="left", on="Store")

# Merge the features data with train and test
train = pd.merge(train, features, how="left", on=["Store", "Date"])
test = pd.merge(test, features, how="left", on=["Store", "Date"])

train.drop(["IsHoliday_y"], axis=1, inplace=True)
test.drop(["IsHoliday_y"], axis=1, inplace=True)

# rename column
train.rename(columns={"IsHoliday_x": "IsHoliday"}, inplace=True)
test.rename(columns={"IsHoliday_x": "IsHoliday"}, inplace=True)

## Datetime features
train["Year"] = train["Date"].dt.year
train["Month"] = train["Date"].dt.month
train["Day"] = train["Date"].dt.day
train["WeekOfYear"] = train["Date"].dt.weekofyear
train["DayOfWeek"] = train["Date"].dt.dayofweek
train["Weekend"] = (train["Date"].dt.weekday >= 5).astype(int)

test["Year"] = test["Date"].dt.year
test["Month"] = test["Date"].dt.month
test["Day"] = test["Date"].dt.day
test["WeekOfYear"] = test["Date"].dt.weekofyear
test["DayOfWeek"] = test["Date"].dt.dayofweek
test["Weekend"] = (test["Date"].dt.weekday >= 5).astype(int)

# convert boolean column to categorical column
train["IsHoliday"] = train["IsHoliday"].map({True: "Yes", False: "No"})
test["IsHoliday"] = test["IsHoliday"].map({True: "Yes", False: "No"})
train["IsHoliday"] = train["IsHoliday"].astype("category")
test["IsHoliday"] = test["IsHoliday"].astype("category")

# ordered the categorical store type col
from pandas.api.types import CategoricalDtype

cat_type = CategoricalDtype(categories=["C", "B", "A"], ordered=True)
train["Type"] = train["Type"].astype(cat_type)
test["Type"] = test["Type"].astype(cat_type)

# convert to categorical columns
train["Store"] = train["Store"].astype("category")
train["Dept"] = train["Dept"].astype("category")
train["Year"] = train["Year"].astype("category")
train["Month"] = train["Month"].astype("category")
train["DayOfWeek"] = train["DayOfWeek"].astype("category")
train["Weekend"] = train["Weekend"].astype("category")

# convert to categorical columns
test["Store"] = test["Store"].astype("category")
test["Dept"] = test["Dept"].astype("category")
test["Year"] = test["Year"].astype("category")
test["Month"] = test["Month"].astype("category")
test["DayOfWeek"] = test["DayOfWeek"].astype("category")
test["Weekend"] = test["Weekend"].astype("category")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.



In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder


# features and labels of train and test set
# labels of test are not provided as we need to predict them

X_train = train.drop(["Weekly_Sales"], axis=1).copy()
y_train = train["Weekly_Sales"].copy()

X_test = test.copy()

# drop and save the date column in a variable
train_date = X_train.pop("Date")
test_date = X_test.pop("Date")


#### Data preparation pipeline

# select numerical and categorical columns
num_cols = X_train.select_dtypes(exclude=["object", "category"]).columns.tolist()
cat_cols = X_train.select_dtypes(include=["object", "category"]).columns.tolist()

# numerical date preprocessing pipeline
num_pipe = make_pipeline(SimpleImputer(strategy="median"), StandardScaler())

# categorical data preprocessing pipeline
cat_pipe = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="NA"),
    OneHotEncoder(handle_unknown="ignore", sparse=False),
)

# full pipeline
full_pipe = ColumnTransformer(
    [("num", num_pipe, num_cols), ("cat", cat_pipe, cat_cols)]
)

In [4]:
X_train_tr = full_pipe.fit_transform(X_train)
X_test_tr = full_pipe.transform(X_test)


# Get the list of categories generated by the one-hot-encoder
ohe_categories = full_pipe.named_transformers_.cat.named_steps.onehotencoder.categories_

# Create nice names for our one hot encoded features
new_ohe_features = [
    f"{col}__{val}" for col, vals in zip(cat_cols, ohe_categories) for val in vals
]

# Create a new list with all names of features
all_features = num_cols + new_ohe_features

# Create pandas dataframe
X_train_tr = pd.DataFrame(X_train_tr, columns=all_features)
X_test_tr = pd.DataFrame(X_test_tr, columns=all_features)

from sklearn.model_selection import train_test_split

X_train_tr, X_valid_tr, y_train, y_valid = train_test_split(X_train_tr,y_train, test_size=0.2, random_state=42)

## Build deep Learning Models

In [5]:
import tensorflow as tf 
from tensorflow import keras

In [6]:
# model = keras.models.Sequential([
#         keras.layers.Dense(64, activation="relu", input_shape=X_train_tr.shape[1:]),
#         keras.layers.Dropout(0.2),
#         keras.layers.Dense(64, activation="relu"),
#         keras.layers.Dropout(0.2),
#         keras.layers.Dense(64, activation="relu"),
#         keras.layers.Dropout(0.2),
#         keras.layers.Dense(1)
# ])

# model.summary()

In [7]:
# # compile and train the model
# model.compile(optimizer="rmsprop", loss= keras.losses.Huber(), metrics=['mae'])
# history = model.fit(X_train_tr, y_train,
#                     validation_data=(X_valid_tr, y_valid),
#                     epochs=25)

In [8]:
# results = pd.DataFrame(history.history)
# results.head()

In [9]:
# fig = go.Figure()
# fig.add_trace(go.Scatter(
#     x=list(range(len(results))), 
#     y= results['loss'], 
#     name="Training Loss" , 
#     mode="lines",
#     line=dict(color="blue")))
# fig.add_trace(go.Scatter(
#     x=list(range(len(results))), 
#     y= results['val_loss'], 
#     name="Validation Loss", 
#     mode="lines",
#     line=dict(color="green")))
# fig.update_layout(title="Training vs Validation Loss",
#                   xaxis=dict(title="Epochs"),
#                   yaxis=dict(title="Huber Loss"))
# fig.show()

In [10]:
def plot_loss(results):
  fig = go.Figure()
  fig.add_trace(go.Scatter(
      x=list(range(len(results))), 
      y= results['loss'], 
      name="Training Loss" , 
      mode="lines",
      line=dict(color="blue")))
  fig.add_trace(go.Scatter(
      x=list(range(len(results))), 
      y= results['val_loss'], 
      name="Validation Loss", 
      mode="lines",
      line=dict(color="green")))
  fig.update_layout(title="Training vs Validation Loss",
                    xaxis=dict(title="Epochs"),
                    yaxis=dict(title="Huber Loss"))
  fig.show()

In [11]:
def plot_mae(results):
  fig = go.Figure()
  fig.add_trace(go.Scatter(
      x=list(range(len(results))), 
      y= results['mae'], 
      name="Training MAE" , 
      mode="lines",
      line=dict(color="blue")))
  fig.add_trace(go.Scatter(
      x=list(range(len(results))), 
      y= results['val_mae'], 
      name="Validation MAE", 
      mode="lines",
      line=dict(color="green")))
  fig.update_layout(title="Training vs Validation MAE",
                    xaxis=dict(title="Epochs"),
                    yaxis=dict(title="Mean Absolute Error"))
  fig.show()


## HyperParameter Tunning

In [24]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV


def build_model(n_hidden=1, n_neurons=32, learning_rate=3e-3, dropout=0.2,input_shape=X_train_tr.shape[1:]):
  model = keras.models.Sequential()
  model.add(keras.layers.InputLayer(input_shape=input_shape))
  for layer in range(n_hidden):
    model.add(keras.layers.Dense(n_neurons, activation="relu"))
    keras.layers.Dropout(dropout)
  model.add(keras.layers.Dense(1))
  optimizer = keras.optimizers.RMSprop(lr=learning_rate)
  model.compile(optimizer=optimizer, loss= keras.losses.Huber(), metrics=['mae'])
  return model

keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)


param_distribs = {
    "n_hidden":[1, 2, 3, 4, 5, 6],
    "n_neurons": np.arange(1,100),
    "learning_rate": reciprocal(3e-4, 3e-2),
    "dropout":[0.1, 0.2, 0.3]
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3)

rnd_search_cv.fit(X_train_tr, y_train, epochs=100,
                  validation_data=(X_valid_tr, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



Epoch 1/100
7027/7027 [==============================] - 24s 3ms/step - loss: 5681.3970 - mae: 5681.9141 - val_loss: 3517.1565 - val_mae: 3517.6545
Epoch 2/100
7027/7027 [==============================] - 21s 3ms/step - loss: 3232.2297 - mae: 3232.7314 - val_loss: 3064.6687 - val_mae: 3065.1687
Epoch 3/100
7027/7027 [==============================] - 21s 3ms/step - loss: 2920.9272 - mae: 2921.4312 - val_loss: 2869.9612 - val_mae: 2870.4604
Epoch 4/100
7027/7027 [==============================] - 21s 3ms/step - loss: 2744.0056 - mae: 2744.5037 - val_loss: 2718.2666 - val_mae: 2718.7659
Epoch 5/100
7027/7027 [==============================] - 21s 3ms/step - loss: 2611.7844 - mae: 2612.2812 - val_loss: 2542.0227 - val_mae: 2542.5222
Epoch 6/100
7027/7027 [==============================] - 21s 3ms/step - loss: 2509.5281 - mae: 2510.0264 - val_loss: 2511.6445 - val_mae: 2512.1440
Epoch 7/100
7027/7027 [==============================] - 21s 3ms/step - loss: 2439.4243 - mae: 2439.9260 - val_l

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



Epoch 1/100
7027/7027 [==============================] - 21s 3ms/step - loss: 5754.7183 - mae: 5755.2236 - val_loss: 3609.4766 - val_mae: 3609.9734
Epoch 2/100
7027/7027 [==============================] - 21s 3ms/step - loss: 3323.8799 - mae: 3324.3831 - val_loss: 3189.5078 - val_mae: 3190.0076
Epoch 3/100
7027/7027 [==============================] - 21s 3ms/step - loss: 2968.9460 - mae: 2969.4475 - val_loss: 2860.9031 - val_mae: 2861.4023
Epoch 4/100
7027/7027 [==============================] - 21s 3ms/step - loss: 2755.5000 - mae: 2756.0005 - val_loss: 2720.3613 - val_mae: 2720.8608
Epoch 5/100
7027/7027 [==============================] - 21s 3ms/step - loss: 2618.4863 - mae: 2618.9827 - val_loss: 2594.0027 - val_mae: 2594.5024
Epoch 6/100
7027/7027 [==============================] - 21s 3ms/step - loss: 2519.2212 - mae: 2519.7219 - val_loss: 2461.8665 - val_mae: 2462.3660
Epoch 7/100
7027/7027 [==============================] - 21s 3ms/step - loss: 2451.7573 - mae: 2452.2532 - val_l

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



...

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



Epoch 1/100
7027/7027 [==============================] - 25s 3ms/step - loss: 4064.7791 - mae: 4065.2739 - val_loss: 2769.4402 - val_mae: 2769.9399
Epoch 2/100
7027/7027 [==============================] - 24s 3ms/step - loss: 2718.3770 - mae: 2718.8826 - val_loss: 3113.9658 - val_mae: 3114.4653
Epoch 3/100
7027/7027 [==============================] - 24s 3ms/step - loss: 2380.3906 - mae: 2380.8933 - val_loss: 2276.2346 - val_mae: 2276.7344
Epoch 4/100
7027/7027 [==============================] - 24s 3ms/step - loss: 2201.8638 - mae: 2202.3640 - val_loss: 2808.4963 - val_mae: 2808.9963
Epoch 5/100
7027/7027 [==============================] - 24s 3ms/step - loss: 2078.5017 - mae: 2078.9980 - val_loss: 2197.9482 - val_mae: 2198.4480
Epoch 6/100
7027/7027 [==============================] - 24s 3ms/step - loss: 1997.7661 - mae: 1998.2693 - val_loss: 1912.0315 - val_mae: 1912.5310
Epoch 7/100
7027/7027 [==============================] - 24s 3ms/step - loss: 1928.1943 - mae: 1928.6964 - val_l

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



10540/10540 [==============================] - 35s 3ms/step - loss: 4661.2700 - mae: 4661.7617 - val_loss: 2927.7227 - val_mae: 2928.2224
Epoch 2/100
10540/10540 [==============================] - 34s 3ms/step - loss: 2498.3064 - mae: 2498.8076 - val_loss: 2230.3440 - val_mae: 2230.8440
Epoch 3/100
10540/10540 [==============================] - 34s 3ms/step - loss: 2127.2610 - mae: 2127.7593 - val_loss: 1995.1874 - val_mae: 1995.6870
Epoch 4/100
10540/10540 [==============================] - 34s 3ms/step - loss: 1971.1987 - mae: 1971.6949 - val_loss: 1996.2921 - val_mae: 1996.7917
Epoch 5/100
10540/10540 [==============================] - 35s 3ms/step - loss: 1881.0580 - mae: 1881.5515 - val_loss: 2371.0771 - val_mae: 2371.5769
Epoch 6/100
10540/10540 [==============================] - 34s 3ms/step - loss: 1819.3152 - mae: 1819.8137 - val_loss: 1875.6112 - val_mae: 1876.1110
Epoch 7/100
10540/10540 [==============================] - 34s 3ms/step - loss: 1768.5088 - mae: 1769.0048 - val

RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7fb5902d8ad0>,
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'dropout': [0.1, 0.2, 0.3],
                                        'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fb5902d87d0>,
                                        'n_hidden': [1, 2, 3, 4, 5, 6],
                                        'n_neurons': array([ 1,  2...
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,

In [25]:
rnd_search_cv.best_score_

-1473.0972900390625

In [26]:
rnd_search_cv.best_params_

{'dropout': 0.1,
 'learning_rate': 0.0004020110173006439,
 'n_hidden': 6,
 'n_neurons': 88}

In [27]:
# make predictions and submission
y_pred = rnd_search_cv.best_estimator_.model.predict(X_test_tr) 
sample_submission["Weekly_Sales"] = y_pred
save_dataframe(sample_submission, "deep_learning_rnd_search.csv")

In [28]:
# save the model
model_path = "/content/drive/MyDrive/workspace/walmart/models"

def save_model(model, model_name, path=model_path):
    path = os.path.join(model_path, model_name)
    model.save(path)

save_model(rnd_search_cv.best_estimator_.model, "dnn_rnd_search.h5")